## [DACON] 진동데이터 활용 충돌체 탐지 AI 경진대회
## 데이터 잘 모르는 사람 (팀명) (Team name)
## 2020년 7월 17일 (제출날짜) (Submission date)

### Part1 CNN Model
### Part2 Wavenet
### Part3 ensembles and post process

# Part1 CNN Model
## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
import math

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import lightgbm as lgb

import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import pairwise_distances
import warnings
from scipy.signal import find_peaks
import os
from sklearn.metrics import mean_absolute_error

import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, Conv1D, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])
def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))
def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04
def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)
def kaeri_metric(y_true, y_pred):
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)
def E1(y_true, y_pred):
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)
def E2(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))
def E2M(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2], np.array(y_pred)[:,2]
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))
def E2V(y_true, y_pred):
    _t, _p = np.array(y_true)[:,3], np.array(y_pred)[:,3]
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [ ]:
def read_data(window_sizes=[[], [], []]):
    train_df = pd.read_csv('../data/train_features.csv')
    target_df = pd.read_csv('../data/train_target.csv')
    test_df = pd.read_csv('../data/test_features.csv')
    train_df = train_df[train_df.index%375<thres].reset_index(drop=True)
    test_df = test_df[test_df.index%375<thres].reset_index(drop=True)

    X_train_l, X_test_l = [], []
    Y_train = target_df.values[:, 1:]

    # making XY
    train = train_df.copy()
    test = test_df.copy()
    for col in ['S1', 'S2', 'S3', 'S4']:
        train[f'speed_{col}'] = train.groupby('id')[col].cumsum().abs().values
        test[f'speed_{col}'] = test.groupby('id')[col].cumsum().abs().values
    train, test = normalize(train, test)
    train = lag_with_pct_change0(train, window_sizes[0])
    test = lag_with_pct_change0(test, window_sizes[0])

    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    # making M
    train = train_df.copy()
    test = test_df.copy()
    for col in ['S1', 'S2', 'S3', 'S4']:
        train[f'speed_{col}'] = train.groupby('id')[col].cumsum().abs().values
        test[f'speed_{col}'] = test.groupby('id')[col].cumsum().abs().values
    train, test = normalize(train, test)
    train = lag_with_pct_change1(train, window_sizes[1])
    test = lag_with_pct_change1(test, window_sizes[1])

    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    # making V
    train = train_df.copy()
    test = test_df.copy()

    train, test = normalize(train, test)
    train = lag_with_pct_change2(train, window_sizes[2])
    test = lag_with_pct_change2(test, window_sizes[2])
    
    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    for i in range(3):
        if i==0:
            print(f'Y_train.shape : {Y_train.shape}')
        print(f'train shape {i} : {X_train_l[i].shape}, test shape {i} : {X_test_l[i].shape}')
    return X_train_l, Y_train, X_test_l

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [ ]:
def lag_with_pct_change0(df, windows=[]):
    for window in [1, 2, 3]:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns='id', inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def lag_with_pct_change1(df, windows=[]):
    for window in [1, 2, 3]:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
        for col in [f'speed_{c}' for c in ['S1', 'S2', 'S3', 'S4']]:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns='id', inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def lag_with_pct_change2(df, windows=[]):
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns=['id'], inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def normalize(train, test):
    if 'S1' in train.columns:
        for col in ['Time', 'S1', 'S2', 'S3', 'S4']:
            train_input_mean = train[col].mean()
            train_input_sigma = train[col].std()
            train[col] = (train[col] - train_input_mean) / train_input_sigma
            test[col] = (test[col] - train_input_mean) / train_input_sigma
    if 'speed_S1' in train.columns:
        for col in ['speed_S1', 'speed_S2', 'speed_S3', 'speed_S4']:
            train_input_mean = train[col].mean()
            train_input_sigma = train[col].std()
            train[col] = (train[col] - train_input_mean) / train_input_sigma
            test[col] = (test[col] - train_input_mean) / train_input_sigma
    return train, test

## 3. 탐색적 자료분석
## Exploratory Data Analysis


## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [ ]:
def set_model(train_target, nf=16):  # 0:x,y, 1:m, 2:v
    activation = 'elu'
    padding = 'valid'
    model = Sequential()
    
    fs = (3,1)

    model.add(Conv2D(nf,fs, padding=padding, activation=activation, input_shape=X_train_l[train_target].shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*2,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*4,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*8,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*16,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*32,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Flatten())
    model.add(Dense(128, activation ='elu'))
    model.add(Dense(64, activation ='elu'))
    model.add(Dense(32, activation ='elu'))
    model.add(Dense(16, activation ='elu'))
    model.add(Dense(4))

    optimizer = keras.optimizers.Adam()

    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1, optimizer=optimizer,)
    else:
        model.compile(loss=my_loss_E2,optimizer=optimizer,)
       
    return model

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [ ]:
%%time
thres=200
N_FOLDS=5
MODEL_SAVE_FOLDER_PATH = ''

SEED=42

ws = [[30], [10, 50], [30]]
X_train_l, Y_train, X_test_l = read_data(window_sizes=ws)

Y_train.shape : (2800, 4)
train shape 0 : (2800, 200, 53, 1), test shape 0 : (700, 200, 53, 1)
train shape 1 : (2800, 200, 97, 1), test shape 1 : (700, 200, 97, 1)
train shape 2 : (2800, 200, 17, 1), test shape 2 : (700, 200, 17, 1)
CPU times: user 2min 26s, sys: 1.03 s, total: 2min 27s
Wall time: 2min 36s


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def plot_error(type_id,pred,true):
    print(pred.shape)

    if type_id == 0:
        _name = 'x_pos'
    elif type_id == 1:
        _name = 'y_pos'
    elif type_id == 2:
        _name = 'mass'
    elif type_id == 3:
        _name = 'velocity'
    elif type_id == 4:
        _name = "distance"
    else:
        _name = 'error'

    x_coord = np.arange(1,pred.shape[0]+1,1)
    if type_id < 2:
        Err_m = (pred[:,type_id] - true[:,type_id])
    elif type_id < 4:
        Err_m = ((pred[:,type_id] - true[:,type_id])/true[:,type_id])*100
    else:
        Err_m = ((pred[:,0]-true[:,0])**2+(pred[:,1]-true[:,1])**2)**0.5

    fig = plt.figure(figsize=(8,6))
    plt.rcParams["font.size"]=15
    plt.scatter(x_coord, Err_m, marker='o')
    plt.title("%s Prediction for Training Data" % _name, size=20)
    plt.xlabel("Data ID", labelpad=10, size=20)
    plt.ylabel("Prediction Error of %s," % _name, labelpad=10, size=20)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.ylim(-100., 100.)
    plt.xlim(0, pred.shape[0]+1)
    plt.show()
    print('std : {}, max : {}, min : {}'.format(np.std(Err_m), np.max(Err_m), np.min(Err_m)))
    return Err_m

def train(model,X,Y,train_target,n_fold=5,fold=0,seed=42):
    best_save = ModelCheckpoint(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=10, min_lr=0.00001, verbose=1)

    kf = KFold(n_splits=n_fold, random_state=seed, shuffle=True)
    splits = [[i,j] for i,j in kf.split(X)]
    tt, tt_target = X[splits[fold][0]], Y[splits[fold][0]]
    vv, vv_target = X[splits[fold][1]], Y[splits[fold][1]]
    
    EPOCHS=500 if train_target==2 else 300
    history = model.fit(tt, tt_target, validation_data=(vv, vv_target),
                  epochs=EPOCHS,
                  batch_size=512,
                  shuffle=True,
                  verbose = 2,
                  callbacks=[es, best_save, reduce_lr], )

    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.axvline(x=np.argmin(history.history['val_loss']), color='b', linewidth=1)
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    
    
    return model
def load_best_model(train_target, fold=0):
    if train_target == 0:
        model = load_model(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', custom_objects={'my_loss_E1': my_loss, })
    else:
        model = load_model(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', custom_objects={'my_loss_E2': my_loss, })

    score = model.evaluate(X_train_l[train_target], Y_train, verbose=0)
    print('loss:', score)

    pred = model.predict(X_train_l[train_target])
    i=0
    print(f'정답(original): {Y_train[i]}, 예측값(original):, {pred[i]}')
    print(f'E1 : {E1(Y_train, pred)}, E2 : {E2(Y_train, pred)}, E2M : {E2M(Y_train, pred)}, E2v : {E2V(Y_train, pred)}')
    
    if train_target ==0:
        plot_error(4,pred,Y_train)
    elif train_target ==1:
        plot_error(2,pred,Y_train)
    elif train_target ==2:
        plot_error(3,pred,Y_train)    
    
    return model, [E1(Y_train, pred), E2(Y_train, pred), E2M(Y_train, pred), E2V(Y_train, pred)]

In [ ]:
if 'submit' not in globals(): submit=pd.read_csv('../data/sample_submission.csv')
if 'metrics' not in globals(): metrics = {}

seed_everything(SEED)
nf_list = [16, 16, 16]
for train_target in range(3):
    fold_metrics = {}
    for k in range(N_FOLDS):
        model = set_model(train_target, nf=nf_list[train_target])
        train(model, X_train_l[train_target], Y_train, train_target, n_fold=N_FOLDS, fold=k, seed=SEED)
        
        best_model, fold_metrics[k] = load_best_model(train_target, k)
        pred_data_test = best_model.predict(X_test_l[train_target])

        if k==0:
            if train_target == 0: # x,y 학습
                submit.iloc[:,1] = pred_data_test[:,0]/N_FOLDS
                submit.iloc[:,2] = pred_data_test[:,1]/N_FOLDS
            elif train_target == 1: # m 학습
                submit.iloc[:,3] = pred_data_test[:,2]/N_FOLDS
            elif train_target == 2: # v 학습
                submit.iloc[:,4] = pred_data_test[:,3]/N_FOLDS
        else:
            if train_target == 0: # x,y 학습
                submit.iloc[:,1] += pred_data_test[:,0]/N_FOLDS
                submit.iloc[:,2] += pred_data_test[:,1]/N_FOLDS
            elif train_target == 1: # m 학습
                submit.iloc[:,3] += pred_data_test[:,2]/N_FOLDS
            elif train_target == 2: # v 학습
                submit.iloc[:,4] += pred_data_test[:,3]/N_FOLDS
        print(f'################################################################################ target : {train_target}, fold : {k} ################################################################################')
    metrics[train_target] = fold_metrics

In [ ]:
cnn_submit = submit.copy()

# Part2 Wavenet
## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
import pandas as pd
import numpy as np

import os
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
!pip install torchcontrib
from torchcontrib.optim import SWA
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import pairwise_distances
# from lookagead import Lookahead
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os
from keras import backend as K
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVR, NuSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
# Any results you write to the current directory are saved as output.

import os
import json
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Using TensorFlow backend.


## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True

def read_data(debar=375):
    train = pd.read_csv('../data/train_features.csv')
    target = pd.read_csv('../data/train_target.csv')
    test = pd.read_csv('../data/test_features.csv')
    sub = pd.read_csv('../data/sample_submission.csv')
    
    train = train[train.index%375<debar].reset_index(drop=True)
    test = test[test.index%375<debar].reset_index(drop=True)

    for col in ['S1', 'S2', 'S3', 'S4']:
        train[f'speed_{col}'] = train.groupby('id')[col].cumsum().abs().values
        test[f'speed_{col}'] = test.groupby('id')[col].cumsum().abs().values
    return train, target, test, sub

def normalize(train, test):
    for col in ['S1', 'S2', 'S3', 'S4']:
        train_input_mean = train[col].mean()
        train_input_sigma = train[col].std()
        train[col] = (train[col] - train_input_mean) / train_input_sigma
        test[col] = (test[col] - train_input_mean) / train_input_sigma

        col = f'speed_{col}'
        train_input_mean = train[col].mean()
        train_input_sigma = train[col].std()
        train[col] = (train[col] - train_input_mean) / train_input_sigma
        test[col] = (test[col] - train_input_mean) / train_input_sigma
    return train, test

def lag_with_pct_change(df, windows):
    for window in [1, 2, 3]:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)

    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values

        for col in ['S1', 'S2', 'S3', 'S4']:
            col = f'speed_{col}'
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values

    for col in ['S1', 'S2', 'S3', 'S4']:
        df.drop(columns=col, inplace=True)
        df.drop(columns=f'speed_{col}', inplace=True)
            
    df = df.replace([np.inf, -np.inf], np.nan)    
    df.fillna(0, inplace=True)
    return df

def run_feat_engineering(df, batch_size):
    df = lag_with_pct_change(df, [10, 50])
    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'id']]#Time
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        # 나중에 group별로 fillna 하는 걸로 바꿔보자
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

def split(GROUP_BATCH_SIZE=4000, SPLITS=5):
    print('Reading Data Started...')
    train, target, test, sub = read_data(debar=GROUP_BATCH_SIZE)
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size=GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size=GROUP_BATCH_SIZE)
    
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')
    print(features)

    train = pd.merge(train, target, how='left', on='id')
    group = train['id']
    kf = GroupKFold(n_splits=SPLITS)
    splits = [x for x in kf.split(train, train[['X', 'Y', 'M', 'V']], group)]
    
    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])
        new_splits.append(new_split)
    target_cols = ['X', 'Y', 'M', 'V']
    
    target = np.array(list(train.groupby('id').apply(lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(list(train.groupby('id').apply(lambda x: x[features].values)))
    
    test = np.array(list(test.groupby('id').apply(lambda x: x[features].values)))

    print(train.shape, test.shape, target.shape)
    return train, test, target, new_splits

## 3. 탐색적 자료분석
## Exploratory Data Analysis


## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [ ]:
class Wave_Block(nn.Module):
    def __init__(self,in_channels,out_channels,dilation_rates):
        super(Wave_Block,self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.convs.append(nn.Conv1d(in_channels,out_channels,kernel_size=1))
        dilation_rates = [2**i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.gate_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=1))
    def forward(self,x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = F.tanh(self.filter_convs[i](x))*F.sigmoid(self.gate_convs[i](x))
            x = self.convs[i+1](x)
            res = torch.add(res, x)
        return res

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        
        # For normal input
        self.wave_block1 = Wave_Block(105,64,12)
        self.bn_1 = nn.BatchNorm1d(64)

        self.wave_block2 = Wave_Block(64,32,8)
        self.bn_2 = nn.BatchNorm1d(32)

        self.wave_block3 = Wave_Block(32,64,4)
        self.bn_3 = nn.BatchNorm1d(64)

        self.wave_block4 = Wave_Block(64,128,1)
        self.bn_4 = nn.BatchNorm1d(128)

        self.fc = nn.Linear(128, 4)

    def flip(self, x, dim):
        dim = x.dim() + dim if dim < 0 else dim
        return x[tuple(slice(None, None) if i != dim
                 else torch.arange(x.size(i)-1, -1, -1).long()
                 for i in range(x.dim()))]
    
    def forward(self,x):
        x = x.permute(0, 2, 1)
        # forward input
        x = self.wave_block1(x)
        x = self.bn_1(x)
        x = self.wave_block2(x)
        x = self.bn_2(x)
        x = self.wave_block3(x)
        x = self.bn_3(x)
        x = self.wave_block4(x)
        x = self.bn_4(x)

        x = torch.mean(x, axis=2)
        x = self.fc(x)

        return x
    
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize
    def load_best_weights(self, model):
        model.load_state_dict(torch.load(self.checkpoint_path))
    def __call__(self, score, model):
        if self.best_score is None or \
                (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
            return 1
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return 2
        return 0

class IronDataset(Dataset):
    def __init__(self, data, labels, training=True, transform=False, seq_len=5000, flip=False, noise_level=0, class_split=0.0):
        
        if flip:
            self.data = np.flip(data)
            self.labels = np.flip(labels)
        else:
            self.data = data
            self.labels = labels.mean(1)
        
        self.transform = transform
        self.training = training
        self.flip = flip
        self.noise_level = noise_level
        self.class_split = class_split
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        data = self.data[idx]
        labels = self.labels[idx]
        return [data.astype(np.float32), labels.astype(np.float32)]

def kaeri_metric(y_true, y_pred):
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)
def E1(y_true, y_pred):
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)
def E2(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

def torch_kaeri_metric(y_true, y_pred):
    return 0.5 * torch_E1(y_true, y_pred) + 0.5 * torch_E2(y_true, y_pred)
def torch_E1(y_true, y_pred):
    _t, _p = y_true[:,:2], y_pred[:,:2]
    return torch.mean(torch.sum(torch.square(_t - _p), axis = 1) / 2e+04)
def torch_E2(y_true, y_pred):
    _t, _p = y_true[:,2:], y_pred[:,2:]
    return torch.mean(torch.sum(torch.square((_t - _p) / (_t + 1e-06)), axis = 1))

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [ ]:
%%time
GROUP_BATCH_SIZE=200
train, test, train_tr, new_splits = split(GROUP_BATCH_SIZE=GROUP_BATCH_SIZE)

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Feature Engineering Completed...
['Time', 'signal_shift_pos_1S1', 'signal_shift_neg_1S1', 'signal_shift_pos_1S2', 'signal_shift_neg_1S2', 'signal_shift_pos_1S3', 'signal_shift_neg_1S3', 'signal_shift_pos_1S4', 'signal_shift_neg_1S4', 'signal_shift_pos_2S1', 'signal_shift_neg_2S1', 'signal_shift_pos_2S2', 'signal_shift_neg_2S2', 'signal_shift_pos_2S3', 'signal_shift_neg_2S3', 'signal_shift_pos_2S4', 'signal_shift_neg_2S4', 'signal_shift_pos_3S1', 'signal_shift_neg_3S1', 'signal_shift_pos_3S2', 'signal_shift_neg_3S2', 'signal_shift_pos_3S3', 'signal_shift_neg_3S3', 'signal_shift_pos_3S4', 'signal_shift_neg_3S4', 'S1_10_mean', 'S1_10_std', 'S1_10_var', 'S1_10_max', 'S1_10_min', 'S2_10_mean', 'S2_10_std', 'S2_10_var', 'S2_10_max', 'S2_10_min', 'S3_10_mean', 'S3_10_std', 'S3_10_var', 'S3_10_max', 'S3_10_min', 'S4_10_mean', 'S4_10_std', 'S4_10_var', 'S4_10_max', 'S4_10_min', 'speed

In [ ]:
NNBATCHSIZE = 256
flip = False
noise = False
LR = 0.0015
EPOCHS = 200
outdir = ''

In [ ]:
seed_everything(42)
test_y = np.zeros([int(262500/GROUP_BATCH_SIZE), GROUP_BATCH_SIZE, 4])
test_dataset = IronDataset(test, test_y, flip=flip)
test_dataloader = DataLoader(test_dataset, NNBATCHSIZE, shuffle=False, num_workers=8, pin_memory=True)

for index, (train_index, val_index, _) in enumerate(new_splits[0:], start=0):
    print("Fold : {}".format(index))
    train_dataset = IronDataset(train[train_index], train_tr[train_index], seq_len=GROUP_BATCH_SIZE, flip=flip, noise_level=False, transform=False)
    train_dataloader = DataLoader(train_dataset, NNBATCHSIZE, shuffle=True, num_workers=8, pin_memory=True)

    valid_dataset = IronDataset(train[val_index], train_tr[val_index], seq_len=GROUP_BATCH_SIZE, flip=flip)
    valid_dataloader = DataLoader(valid_dataset, NNBATCHSIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = Regressor()
    model = model.cuda()

    early_stopping = EarlyStopping(patience=50, is_maximize=False, checkpoint_path=os.path.join(outdir, f'fold_{index}.pt'))
    
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=0.000025)
    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=5, factor=0.9)

    criterion = nn.L1Loss()
    score = 10000000

    for epoch in range(EPOCHS):
        train_loss, valid_loss = 0, 0
        custom_trn_loss, custom_val_loss = 0, 0

        model.train()  # prep model for training

        for (x, y) in train_dataloader:
            x = x.cuda()
            y = y.cuda()
            
            opt.zero_grad()
            predictions = model(x)
            loss = criterion(predictions, y)
            custom = torch_kaeri_metric(y, predictions)
            loss = loss + custom*2

            loss.backward()
            opt.step()
            custom_trn_loss += custom
        custom_trn_loss = custom_trn_loss / len(train_dataloader)

        # prep model for evaluation
        model.eval()  
        with torch.no_grad():
            for (x, y) in valid_dataloader:
                x = x.cuda()
                y = y.cuda()
                
                predictions = model(x)
                loss = criterion(predictions, y)
                custom = torch_kaeri_metric(y, predictions).item()
                
                valid_loss += loss
                custom_val_loss += custom
        
        valid_loss = valid_loss / len(valid_dataloader)
        custom_val_loss = custom_val_loss / len(valid_dataloader)
        
        early_loss = valid_loss + custom_val_loss*2
        res = early_stopping(early_loss, model)
        
        if res==2:
            print("Early Stopping")
            print('folder %d global best val max f1 model score %f' % (index, early_stopping.best_score))
            print ("Epoch [{}/{}], val_loss : {:.4f}, tr_cus_loss : {:.4f}, va_cus_loss : {:.4f}, lr : {:.6f}, early_loss : {:.4f}".
                    format(epoch+1, EPOCHS, valid_loss, custom_trn_loss, custom_val_loss, opt.param_groups[0]['lr'], early_loss))
            break
        elif res==1:
            if score>early_loss:
                score = early_loss
                print ("Epoch [{}/{}], val_loss : {:.4f}, tr_cus_loss : {:.4f}, va_cus_loss : {:.4f}, lr : {:.6f}, early_loss : {:.4f}".
                    format(epoch+1, EPOCHS, valid_loss, custom_trn_loss, custom_val_loss, opt.param_groups[0]['lr'], early_loss))
        
        # 플레쳐
        schedular.step(early_loss)

In [ ]:
seed_everything(42)
n_folds = 5
test_y = np.zeros([700, 4])
test_dataset = IronDataset(test, test_y, flip=flip)
test_dataloader = DataLoader(test_dataset, NNBATCHSIZE, shuffle=False, num_workers=8, pin_memory=True)

oof_mae = 0
oof_custom = 0

for index, (train_index, val_index, _) in enumerate(new_splits[0:], start=0):
    print("Fold : {}".format(index))
    valid_dataset = IronDataset(train[val_index], train_tr[val_index], seq_len=GROUP_BATCH_SIZE, flip=flip)
    valid_dataloader = DataLoader(valid_dataset, NNBATCHSIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = Regressor()
    model = model.cuda()
    model.load_state_dict(torch.load(outdir + f'fold_{index}.pt'))

    model.eval()  # prep model for evaluation
    oof = []
    with torch.no_grad():
        for (x, y) in valid_dataloader:
            x = x.cuda()
            y = y.cuda()
            
            pred = model(x)
            pred = pred.detach().cpu().numpy()
            oof += [pred]

    with torch.no_grad():
        for i, (x, y) in enumerate(test_dataloader):
            x = x.cuda()
            y = y.cuda()

            predictions = model(x)
            predictions = predictions.detach().cpu().numpy()

            test_y[NNBATCHSIZE*i:NNBATCHSIZE*(i+1), :] += predictions / n_folds

    valid_oof = np.concatenate(oof)
    valid_ture = train_tr[val_index].mean(1)
    print('mae : {:.6f}, custom_metirc : {:.6f}'.format(mean_absolute_error(valid_ture, valid_oof), kaeri_metric(valid_ture, valid_oof)))
    
    oof_mae += mean_absolute_error(valid_ture, valid_oof) / n_folds
    oof_custom += kaeri_metric(valid_ture, valid_oof) / n_folds
print(oof_mae, oof_custom)

In [ ]:
submit.iloc[:, 1:] = test_y
wavenet_submit = submit.copy()

# Part3 CNN Model Pseudo
## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
import math

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import lightgbm as lgb

import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import pairwise_distances
import warnings
from scipy.signal import find_peaks
import os
from sklearn.metrics import mean_absolute_error

import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, Conv1D, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])
def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))
def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04
def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)
def kaeri_metric(y_true, y_pred):
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)
def E1(y_true, y_pred):
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)
def E2(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))
def E2M(y_true, y_pred):
    _t, _p = np.array(y_true)[:,2], np.array(y_pred)[:,2]
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))
def E2V(y_true, y_pred):
    _t, _p = np.array(y_true)[:,3], np.array(y_pred)[:,3]
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [ ]:
def read_data(window_sizes=[[], [], []]):
    train_df = pd.read_csv('../data/train_features.csv')
    target_df = pd.read_csv('../data/train_target.csv')
    test_df = pd.read_csv('../data/test_features.csv')
    train_df = train_df[train_df.index%375<thres].reset_index(drop=True)
    test_df = test_df[test_df.index%375<thres].reset_index(drop=True)

    X_train_l, X_test_l = [], []
    Y_train = target_df.values[:, 1:]

    # making XY
    train = train_df.copy()
    test = test_df.copy()
    for col in ['S1', 'S2', 'S3', 'S4']:
        train[f'speed_{col}'] = train.groupby('id')[col].cumsum().abs().values
        test[f'speed_{col}'] = test.groupby('id')[col].cumsum().abs().values
    train, test = normalize(train, test)
    train = lag_with_pct_change0(train, window_sizes[0])
    test = lag_with_pct_change0(test, window_sizes[0])

    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    # making M
    train = train_df.copy()
    test = test_df.copy()
    for col in ['S1', 'S2', 'S3', 'S4']:
        train[f'speed_{col}'] = train.groupby('id')[col].cumsum().abs().values
        test[f'speed_{col}'] = test.groupby('id')[col].cumsum().abs().values
    train, test = normalize(train, test)
    train = lag_with_pct_change1(train, window_sizes[1])
    test = lag_with_pct_change1(test, window_sizes[1])

    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    # making V
    train = train_df.copy()
    test = test_df.copy()

    train, test = normalize(train, test)
    train = lag_with_pct_change2(train, window_sizes[2])
    test = lag_with_pct_change2(test, window_sizes[2])
    
    X_train = train.values.reshape((2800,thres,-1,1))
    X_test = test.values.reshape((700,thres,-1,1))
    X_train_l += [X_train]
    X_test_l += [X_test]

    for i in range(3):
        if i==0:
            print(f'Y_train.shape : {Y_train.shape}')
        print(f'train shape {i} : {X_train_l[i].shape}, test shape {i} : {X_test_l[i].shape}')
    return X_train_l, Y_train, X_test_l

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [ ]:
def lag_with_pct_change0(df, windows=[]):
    for window in [1, 2, 3]:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns='id', inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def lag_with_pct_change1(df, windows=[]):
    for window in [1, 2, 3]:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
        for col in [f'speed_{c}' for c in ['S1', 'S2', 'S3', 'S4']]:
            df['signal_shift_pos_' + str(window) + col] = df.groupby('id')[col].shift(window).fillna(0)
            df['signal_shift_neg_' + str(window) + col] = df.groupby('id')[col].shift(-1 * window).fillna(0)
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_std'] = df.groupby('id')[col].rolling(window=window).std().values
            df[f'{col}_{window}_var'] = df.groupby('id')[col].rolling(window=window).var().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns='id', inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def lag_with_pct_change2(df, windows=[]):
    for window in windows:
        for col in ['S1', 'S2', 'S3', 'S4']:
            df[f'{col}_{window}_mean'] = df.groupby('id')[col].rolling(window=window).mean().values
            df[f'{col}_{window}_max'] = df.groupby('id')[col].rolling(window=window).max().values
            df[f'{col}_{window}_min'] = df.groupby('id')[col].rolling(window=window).min().values
    df.drop(columns=['id'], inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.fillna(0, inplace=True)
    return df
##############################################################################################################################
def normalize(train, test):
    if 'S1' in train.columns:
        for col in ['Time', 'S1', 'S2', 'S3', 'S4']:
            train_input_mean = train[col].mean()
            train_input_sigma = train[col].std()
            train[col] = (train[col] - train_input_mean) / train_input_sigma
            test[col] = (test[col] - train_input_mean) / train_input_sigma
    if 'speed_S1' in train.columns:
        for col in ['speed_S1', 'speed_S2', 'speed_S3', 'speed_S4']:
            train_input_mean = train[col].mean()
            train_input_sigma = train[col].std()
            train[col] = (train[col] - train_input_mean) / train_input_sigma
            test[col] = (test[col] - train_input_mean) / train_input_sigma
    return train, test

## 3. 탐색적 자료분석
## Exploratory Data Analysis


## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [ ]:
def set_model(train_target, nf=16):  # 0:x,y, 1:m, 2:v
    activation = 'elu'
    padding = 'valid'
    model = Sequential()
    
    fs = (3,1)

    model.add(Conv2D(nf,fs, padding=padding, activation=activation, input_shape=X_train_l[train_target].shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*2,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*4,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*8,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*16,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*32,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Flatten())
    model.add(Dense(128, activation ='elu'))
    model.add(Dense(64, activation ='elu'))
    model.add(Dense(32, activation ='elu'))
    model.add(Dense(16, activation ='elu'))
    model.add(Dense(4))

    optimizer = keras.optimizers.Adam()

    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1, optimizer=optimizer,)
    else:
        model.compile(loss=my_loss_E2,optimizer=optimizer,)
       
    return model

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [ ]:
%%time
thres=200
N_FOLDS=5
MODEL_SAVE_FOLDER_PATH = ''

SEED=42

ws = [[30], [10, 50], [30]]
X_train_l, Y_train, X_test_l = read_data(window_sizes=ws)

Y_train.shape : (2800, 4)
train shape 0 : (2800, 200, 53, 1), test shape 0 : (700, 200, 53, 1)
train shape 1 : (2800, 200, 97, 1), test shape 1 : (700, 200, 97, 1)
train shape 2 : (2800, 200, 17, 1), test shape 2 : (700, 200, 17, 1)
CPU times: user 3min 31s, sys: 1.25 s, total: 3min 32s
Wall time: 3min 36s


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def plot_error(type_id,pred,true):
    print(pred.shape)

    if type_id == 0:
        _name = 'x_pos'
    elif type_id == 1:
        _name = 'y_pos'
    elif type_id == 2:
        _name = 'mass'
    elif type_id == 3:
        _name = 'velocity'
    elif type_id == 4:
        _name = "distance"
    else:
        _name = 'error'

    x_coord = np.arange(1,pred.shape[0]+1,1)
    if type_id < 2:
        Err_m = (pred[:,type_id] - true[:,type_id])
    elif type_id < 4:
        Err_m = ((pred[:,type_id] - true[:,type_id])/true[:,type_id])*100
    else:
        Err_m = ((pred[:,0]-true[:,0])**2+(pred[:,1]-true[:,1])**2)**0.5

    fig = plt.figure(figsize=(8,6))
    plt.rcParams["font.size"]=15
    plt.scatter(x_coord, Err_m, marker='o')
    plt.title("%s Prediction for Training Data" % _name, size=20)
    plt.xlabel("Data ID", labelpad=10, size=20)
    plt.ylabel("Prediction Error of %s," % _name, labelpad=10, size=20)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.ylim(-100., 100.)
    plt.xlim(0, pred.shape[0]+1)
    plt.show()
    print('std : {}, max : {}, min : {}'.format(np.std(Err_m), np.max(Err_m), np.min(Err_m)))
    return Err_m

def train(model,X,Y,train_target,n_fold=5,fold=0,seed=42):
    best_save = ModelCheckpoint(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=10, min_lr=0.00001, verbose=1)

    kf = KFold(n_splits=n_fold, random_state=seed, shuffle=True)
    splits = [[i,j] for i,j in kf.split(X)]
    tt, tt_target = X[splits[fold][0]], Y[splits[fold][0]]
    vv, vv_target = X[splits[fold][1]], Y[splits[fold][1]]
    
    # pseudo labeling
    splits2 = [[i,j] for i,j in kf.split(X_test_l[train_target])]
    tt = np.concatenate([tt, X_test_l[train_target][splits2[fold][1]]])
    tt_target = np.concatenate([tt_target, temp_target[splits2[fold][1]]])

    EPOCHS=500 if train_target==2 else 300
    history = model.fit(tt, tt_target, validation_data=(vv, vv_target),
                  epochs=EPOCHS,
                  batch_size=512,
                  shuffle=True,
                  verbose = 2,
                  callbacks=[es, best_save, reduce_lr], )

    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.axvline(x=np.argmin(history.history['val_loss']), color='b', linewidth=1)
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    
    
    return model
def load_best_model(train_target, fold=0):
    if train_target == 0:
        model = load_model(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', custom_objects={'my_loss_E1': my_loss, })
    else:
        model = load_model(MODEL_SAVE_FOLDER_PATH + f'best_m{train_target}_{fold}.hdf5', custom_objects={'my_loss_E2': my_loss, })

    score = model.evaluate(X_train_l[train_target], Y_train, verbose=0)
    print('loss:', score)

    pred = model.predict(X_train_l[train_target])
    i=0
    print(f'정답(original): {Y_train[i]}, 예측값(original):, {pred[i]}')
    print(f'E1 : {E1(Y_train, pred)}, E2 : {E2(Y_train, pred)}, E2M : {E2M(Y_train, pred)}, E2v : {E2V(Y_train, pred)}')
    
    if train_target ==0:
        plot_error(4,pred,Y_train)
    elif train_target ==1:
        plot_error(2,pred,Y_train)
    elif train_target ==2:
        plot_error(3,pred,Y_train)    
    
    return model, [E1(Y_train, pred), E2(Y_train, pred), E2M(Y_train, pred), E2V(Y_train, pred)]

In [ ]:
# pseudo label
cnn_submit[['X', 'Y']] = cnn_submit[['X', 'Y']].clip(-400, 400)
wavenet_submit[['X', 'Y']] = wavenet_submit[['X', 'Y']].clip(-400, 400)
temp_target = (wavenet_submit/2 + cnn_submit/2).values[:, 1:]

In [ ]:
submit = pd.read_csv('../data/sample_submission.csv')
metrics = {}

seed_everything(SEED)
nf_list = [16, 16, 16]
for train_target in range(0,1):
    fold_metrics = {}
    for k in range(N_FOLDS):
        model = set_model(train_target, nf=nf_list[train_target])
        train(model, X_train_l[train_target], Y_train, train_target, n_fold=N_FOLDS, fold=k, seed=SEED)
        
        best_model, fold_metrics[k] = load_best_model(train_target, k)
        pred_data_test = best_model.predict(X_test_l[train_target])

        if k==0:
            if train_target == 0: # x,y 학습
                submit.iloc[:,1] = pred_data_test[:,0]/N_FOLDS
                submit.iloc[:,2] = pred_data_test[:,1]/N_FOLDS
            elif train_target == 1: # m 학습
                submit.iloc[:,3] = pred_data_test[:,2]/N_FOLDS
            elif train_target == 2: # v 학습
                submit.iloc[:,4] = pred_data_test[:,3]/N_FOLDS
        else:
            if train_target == 0: # x,y 학습
                submit.iloc[:,1] += pred_data_test[:,0]/N_FOLDS
                submit.iloc[:,2] += pred_data_test[:,1]/N_FOLDS
            elif train_target == 1: # m 학습
                submit.iloc[:,3] += pred_data_test[:,2]/N_FOLDS
            elif train_target == 2: # v 학습
                submit.iloc[:,4] += pred_data_test[:,3]/N_FOLDS
        print(f'################################################################################ target : {train_target}, fold : {k} ################################################################################')
    metrics[train_target] = fold_metrics

## 6. 결과 및 결언
## Conclusion & Discussion

In [ ]:
submit[['X', 'Y']] = submit[['X', 'Y']].clip(-400, 400)
submit['M'] = temp_target[:, 2] + 1.8
submit['V'] = temp_target[:, 3] + 0.0207

In [ ]:
submit.to_csv('submission.csv', index=False)